In [1]:
import sys, os
import re
import pandas as pd
import itertools, nltk, string 

import re
import nltk
import string
from nltk.stem import WordNetLemmatizer 

lemmatizer = WordNetLemmatizer() 

nltk_words = set(nltk.corpus.stopwords.words('english'))
stop_words = []
for word in nltk_words:
    stop_words.append(word.translate(str.maketrans('', '', string.punctuation)))

def preprocess(sentence):
    res = sentence.lower()
    res = res.translate(str.maketrans('', '', string.punctuation))
    tokenized_words = nltk.word_tokenize(res)
    res = [word for word in tokenized_words if word not in stop_words]
    res = [lemmatizer.lemmatize(r) for r in res]
    res = [re.sub(r"[^A-Za-z]+", '', r) for r in res]
    return res

positive_lexicon = []
negative_lexicon = []

def read_lexicon():
    global positive_lexicon;
    global negative_lexicon;
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'positive-words.txt'), 'r') as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
         
        positive_lexicon = file.readlines()
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'negative-words.txt'), 'r', encoding = "ISO-8859-1") as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
        
        negative_lexicon = file.readlines()
        
    positive_lexicon = list(map(lambda word: word.rstrip("\n\r"), positive_lexicon))
    negative_lexicon = list(map(lambda word: word.rstrip("\n\r"), negative_lexicon))    
        
read_lexicon()
opinion_lexicon = positive_lexicon + negative_lexicon

In [2]:
import os
os.environ["CORENLP_HOME"] = r'C:\stanford-corenlp-full-2018-10-05'

import corenlp 
client = corenlp.CoreNLPClient()

to_be = ['be', 'am', 'is', 'are', 'was', 'were', 'been', 'being']

def get_adjective(sentence):
    text = sentence.lower()
    res = []
    try:
        pattern = '[{pos:JJ}] | [{pos:JJR}] | [{pos:JJS}]'
        matches = client.tokensregex(sentence, pattern)
        response = matches['sentences'][0]
        len_chunk = response['length']
        if len_chunk > 0:
            for index in range(0, len_chunk):
                res.append(response[str(index)]['text'])
    except:
        print('err : ', sentence)
        res = []
    return res

def get_verb(sentence):
    text = sentence.lower()
    res = []
    try:
        pattern = '[{pos:/VB.*/}]'
        matches = client.tokensregex(sentence, pattern)
        response = matches['sentences'][0]
        len_chunk = response['length']
        if len_chunk > 0:
            for index in range(0, len_chunk):
                word = response[str(index)]['text']
                if word not in to_be:
                    res.append(word)
    except:
        print('err : ', sentence)
        res = []
    return res

def get_op_word(middle, opinions):
    temp = 9999
    res = ''
    for word, index in opinions:
        jarak = abs(index - middle)
        if jarak <= temp:
            res = word
            temp = jarak
    return res

In [5]:
import string

def midpoint(p1, p2):
    return int((p1+p2)/2)

def rem_punct(word):
    return word.translate(str.maketrans('', '', string.punctuation))

def create_file(category):
    sf = pd.DataFrame(columns = ['id','review', 'opinion', 'label', 'preprocessed'])
    buang = 0
    for index in range(0, len(df)):
        if type(df['predict'][index]) == float:
            continue
        labels = df['label'][index].split(',')
        predicts = df['predict'][index].split('|')
        if category in labels and category in predicts:
            kata_kata = []
            tokens = df['review'][index].replace('-',' ').replace('  ', ' ').strip().lower().split(' ')
            terms = df['term'][index].split('|')
            if "neutral" in df['polarity'][index]:
#                 print('neutral')
                continue
            
            polarities = df['polarity'][index].split(',')

            new_polar = []
            for x, label in enumerate(labels):
                if category == label:
                    new_polar.append(polarities[x])
            
            new_terms = []
            for x, predict in enumerate(predicts):
                if category == predict:
                    new_terms.append(terms[x])
            
            for term in new_terms:
                i = 0
                for token in tokens:
                    if term in token:
                        kata_kata.append(i)
                    i += 1
            middle = midpoint(min(kata_kata), max(kata_kata))
            
            opinion_pos = []
            i = 0
            for token in tokens:
                if rem_punct(token) in opinion_lexicon:
                    opinion_pos.append((rem_punct(token), i))
                i += 1
            
            opinion_adj = []
            if len(opinion_pos) == 0:
                opinion_adj = get_adjective(df['review'][index].replace('-',' ').replace('  ', ' ').strip().lower())
                i = 0
                for token in tokens:
                    for op in opinion_adj:
                        if op in token:
                            opinion_pos.append((rem_punct(token), i))
                    i += 1
                    
            
            opinion_vb = []
            if len(opinion_pos) == 0:
                opinion_vb = get_verb(df['review'][index].replace('-',' ').replace('  ', ' ').strip().lower())
                i = 0
                for token in tokens:
                    for op in opinion_vb:
                        if op in token:
                            opinion_pos.append((rem_punct(token), i))
                    i += 1
                    
            final_opinion = get_op_word(middle, opinion_pos)
            
            label_polar = ''
            cp, cn = 0, 0
            for polar in new_polar:
                if polar == 'positive':
                    cp += 1
                elif polar == 'negative':
                    cn += 1
            if cp > cn:
                label_polar = 'positive'
            elif cp < cn:
                label_polar = 'negative'
            else:
                buang += 1
                continue
            
            sf = sf.append({'id': df['id'][index], 
                    'review': df['review'][index],
                    'opinion': final_opinion,
                    'label': label_polar,
                    'preprocessed': preprocess(df['review'][index]),
                   }, ignore_index=True)
    print(buang)
    sf.to_csv("Results/"+ category +".csv")
    sf.to_excel("Results/"+ category +".xlsx")

In [6]:
import pandas as pd

df = pd.read_csv('Results/Categorization/lda2vec-partial-ner-elmo-money.csv')
# df.head()

In [7]:
create_file('FOOD')

err :  the signs, the specials menus, food, and even all the waitstaff are all totally japanese.
7


In [8]:
create_file('AMBIENCE')

1


In [9]:
create_file('SERVICE')

3


In [10]:
create_file('PRICES')

1


In [21]:
from nltk.parse.corenlp import CoreNLPParser
st = CoreNLPParser()
tokenized_sent = list(st.tokenize('the pizza and wine were excellent-the service too--but what really made this place was the backyard dining area.'))
tokenized_sent


['the',
 'pizza',
 'and',
 'wine',
 'were',
 'excellent-the',
 'service',
 'too',
 '--',
 'but',
 'what',
 'really',
 'made',
 'this',
 'place',
 'was',
 'the',
 'backyard',
 'dining',
 'area',
 '.']